In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12150860787051154919
]


In [3]:
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.preprocessing import text
import keras
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras import utils

config = tf.ConfigProto( device_count = {'GPU': 3000 , 'CPU': 2} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)
print("Tensorflow version", tf.__version__)

Using TensorFlow backend.


Tensorflow version 1.8.0


In [4]:
data = pd.read_csv(r'E:\Study\Sabertooth Investments\News And Quant\News\combined.csv',encoding = 'ISO-8859-1')

In [5]:
data.columns

Index(['Label', 'Title'], dtype='object')

In [6]:
from sklearn.utils import shuffle
data = shuffle(data)

In [7]:
train_size = int(len(data)*.80)
print('Train size:', train_size)

Train size: 3640


In [8]:
train_sentence1 = data['Title'][:train_size]
train_sentence1 = [x for x in train_sentence1]

train_posts1 = data['Title'][:train_size]

train_tags1 = data['Label'][:train_size]  

# train_sentence1 = []
# train_tags1 = []
# train_posts1 = []
# train_time1=[]

# for i in range(len(train_tags)):
#     if train_tags[i] < -0.3:
#         train_sentence1.append(train_sentence[i])
#         train_tags1.append(-1)
#         train_posts1.append(str(train_posts[i]))
#         train_time1.append(train_time[i])
#     elif train_tags[i] >0.3:
#         train_sentence1.append(train_sentence[i])
#         train_tags1.append(1)
#         train_posts1.append(str(train_posts[i]))
#         train_time1.append(train_time[i])

test_sentence1 = data['Title'][train_size:]
test_sentence1 = [x for x in test_sentence1]

test_posts1 = data['Title'][train_size:]

test_tags1 = data['Label'][train_size:]

# for i in range(len(test_tags)):
#     if test_tags[i] < -0.3:
#         test_sentence1.append(test_sentence[i])
#         test_tags1.append(-1)
#         test_posts1.append(str(test_posts[i]))
#         test_time1.append(test_time[i])
#     elif test_tags[i] >0.3:
#         test_sentence1.append(test_sentence[i])
#         test_tags1.append(1)
#         test_posts1.append(str(test_posts[i]))
#         test_time1.append(test_time[i])
        
for x in test_tags1[:10]:
    print (x )
    
print (len(test_tags1),len(train_tags1),sep="\n")
print(test_tags1)

1
2
0
0
0
0
2
0
2
0
910
3640
746     1
2523    2
366     0
4369    0
1267    0
401     0
2555    2
1179    0
1386    2
4430    0
770     1
4466    0
104     0
1397    2
3925    2
421     0
2472    2
4173    0
3364    0
3011    0
2233    1
2965    2
4334    1
2575    0
2678    2
2839    0
3086    2
3472    1
4388    2
3024    0
       ..
1555    2
64      0
56      0
2       0
2532    1
3147    2
2453    1
1368    0
207     0
3875    2
2378    1
2041    0
1449    1
1131    0
3543    2
1681    0
392     0
3169    2
2685    2
535     1
1865    0
1539    2
205     2
3227    0
3739    0
4378    2
3241    1
3065    1
2022    2
4017    0
Name: Label, Length: 910, dtype: int64


In [9]:
for i in test_tags1[:10]:
    print(i)
max_words = 3000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

1
2
0
0
0
0
2
0
2
0


In [10]:
tokenize.fit_on_texts(train_posts1)
x_train = tokenize.texts_to_matrix(train_posts1)
x_test = tokenize.texts_to_matrix(test_posts1)

In [11]:
encoder = LabelEncoder()
encoder.fit(train_tags1)
y_train = encoder.transform(train_tags1)
y_test = encoder.transform(test_tags1)

In [12]:
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [13]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
batch_size = 32
### Build the model
model = Sequential()
model.add(Dense(512,input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])


x_train shape: (3640, 3000)
x_test shape: (910, 3000)
y_train shape: (3640, 3)
y_test shape: (910, 3)


In [14]:
history = model.fit(x_train,y_train,
                   batch_size=batch_size,
                   epochs=200,
                   verbose=1,
                   validation_split=0.2)

Train on 2912 samples, validate on 728 samples
Epoch 1/200
2912/2912 [==============================] - 4s 1ms/step - loss: 1.0824 - acc: 0.4141 - val_loss: 1.0821 - val_acc: 0.4217
Epoch 2/200
2912/2912 [==============================] - 3s 1ms/step - loss: 1.0217 - acc: 0.4849 - val_loss: 1.0896 - val_acc: 0.4052
Epoch 3/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.9474 - acc: 0.5677 - val_loss: 1.1074 - val_acc: 0.4080
Epoch 4/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.8561 - acc: 0.6387 - val_loss: 1.1319 - val_acc: 0.3970
Epoch 5/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.7664 - acc: 0.6899 - val_loss: 1.1803 - val_acc: 0.3832
Epoch 6/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.6781 - acc: 0.7328 - val_loss: 1.2348 - val_acc: 0.3997
Epoch 7/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.6090 - acc: 0.7593 - val_loss: 1.2832 - val_acc: 0.4093
Epoch 8/

Epoch 61/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.1048 - acc: 0.9609 - val_loss: 2.9801 - val_acc: 0.3887
Epoch 62/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0985 - acc: 0.9629 - val_loss: 2.9904 - val_acc: 0.3887
Epoch 63/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.1043 - acc: 0.9636 - val_loss: 3.0137 - val_acc: 0.3915
Epoch 64/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0973 - acc: 0.9612 - val_loss: 3.0291 - val_acc: 0.3970
Epoch 65/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.1023 - acc: 0.9591 - val_loss: 3.0263 - val_acc: 0.3997
Epoch 66/200
2912/2912 [==============================] - 4s 1ms/step - loss: 0.1029 - acc: 0.9643 - val_loss: 3.0519 - val_acc: 0.3942
Epoch 67/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.1106 - acc: 0.9602 - val_loss: 3.0798 - val_acc: 0.3956
Epoch 68/200
2912/2912 [========================

2912/2912 [==============================] - 3s 1ms/step - loss: 0.0786 - acc: 0.9643 - val_loss: 3.8026 - val_acc: 0.3860
Epoch 122/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0689 - acc: 0.9688 - val_loss: 3.7961 - val_acc: 0.3819
Epoch 123/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0759 - acc: 0.9684 - val_loss: 3.8149 - val_acc: 0.3819
Epoch 124/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0737 - acc: 0.9650 - val_loss: 3.8492 - val_acc: 0.3805
Epoch 125/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0781 - acc: 0.9639 - val_loss: 3.8677 - val_acc: 0.3805
Epoch 126/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0725 - acc: 0.9653 - val_loss: 3.8672 - val_acc: 0.3887
Epoch 127/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0741 - acc: 0.9674 - val_loss: 3.8705 - val_acc: 0.3860
Epoch 128/200
2912/2912 [==============================

2912/2912 [==============================] - 3s 1ms/step - loss: 0.0613 - acc: 0.9691 - val_loss: 4.4979 - val_acc: 0.3764
Epoch 181/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0611 - acc: 0.9674 - val_loss: 4.5096 - val_acc: 0.3750
Epoch 182/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0656 - acc: 0.9698 - val_loss: 4.5019 - val_acc: 0.3819
Epoch 183/200
2912/2912 [==============================] - 3s 999us/step - loss: 0.0666 - acc: 0.9674 - val_loss: 4.4904 - val_acc: 0.3791
Epoch 184/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0610 - acc: 0.9705 - val_loss: 4.4850 - val_acc: 0.3764
Epoch 185/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0617 - acc: 0.9667 - val_loss: 4.5086 - val_acc: 0.3791
Epoch 186/200
2912/2912 [==============================] - 3s 1ms/step - loss: 0.0614 - acc: 0.9701 - val_loss: 4.5339 - val_acc: 0.3805
Epoch 187/200
2912/2912 [============================

In [15]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

910/910 [==============================] - 0s 155us/step
Test score: 4.418216324900532
Test accuracy: 0.3758241758569256


In [18]:
text_labels = encoder.classes_ 
for i in range(910):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print('Actual Title: '+test_sentence1[i])
    print('Actual label:' + str(test_tags1[i-1:i]))
    print("Predicted label: " + str(predicted_label) + "\n")

Actual Title: Buy Infosys; target of Rs 1265: Edelweiss
Actual label:Series([], Name: Label, dtype: int64)
Predicted label: 0

Actual Title: RIL up 1%; Reliance Jio launches 4G service
Actual label:746    1
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Infosys plans to invest in Silicon Valley fund to track innovative startups']
Actual label:2523    2
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Buy Tata Steel; target of Rs 612: Firstcall Research
Actual label:366    0
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Infosys to acquire London-based Brilliant Basics
Actual label:4369    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: ['Buy Infosys with target at Rs 1230: AK Prabhakar']
Actual label:1267    0
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Reliance at new 52-week low
Actual label:401    0
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Sensex, Nifty flat; Tata Motors gain, ITC & Infosys fall


Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Tata steel rallies over 2% on talks to sell long steel business in Europe']
Actual label:3754    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Tata Steel at its 27-month high; Q3 results show extraordinary improvement
Actual label:3911    1
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Sensex, Nifty flat; Axis Bank, Tata Motors gain 1-2%
Actual label:4414    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Tata Steel stock surges 5%
Actual label:3493    1
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Buy Infosys with a target of Rs 4425: Prakash Gaba']
Actual label:4518    1
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Vodafone knocks Delhi High CourtÃ¢??s door against Jio free voice services
Actual label:202    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Reliance hits record high; lifts Nifty and Sensex from day?s low
Actual label:24

Actual Title: Top 19 stocks in focus today: Tata Motors, J Kumar Infra, Biocon
Actual label:347    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Scaling up gradually! Tata Steel stock up 2%
Actual label:3684    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Sensex, Nifty slip 1%; ONGC, Sun Pharma, Infosys outperform
Actual label:4498    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Tata group companies lose over Rs 26,000 cr in mcap in 3 days
Actual label:1141    2
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['MD Ranganath to take charge as Infosys CFO as Rajiv Bansal quits']
Actual label:3293    2
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Reliance Defence and Emirates Defence Industries signs MOU for strategic partnership in defence sector
Actual label:336    1
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Tata Motors gains after Jaguar Land Rover posts decent retail sales in October
Actual 

Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Tata Motors surges over 3% as JLR US sales hit record high in 2016']
Actual label:1022    2
Name: Label, dtype: int64
Predicted label: 0

Actual Title: TATA Steel down 5%; to commence commercial steel production from Kalinganagar plant
Actual label:2991    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: ["New twist in Infosys saga, firm's board members offer to resign"]
Actual label:4478    0
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Buy Infosys with a target of Rs 3560: Ashwani Gujral']
Actual label:666    1
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Tata Motors slips over 1% on lower global sales data for September']
Actual label:135    2
Name: Label, dtype: int64
Predicted label: 1

Actual Title: ['Share sale by Infosys founders could see Narayana Murthy, Nandan Nilekani come together for philanthropy']
Actual label:2758    2
Name: Label, dtype: int64
Predicted label: 1

Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['After the Q2 show, should you shift from TCS to Infosys?']
Actual label:4412    1
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Bullish Tata Motors despite weak Q4 earnings: Prabhudas
Actual label:186    1
Name: Label, dtype: int64
Predicted label: 1

Actual Title: ['Tata Motors surges 3% on better top line growth']
Actual label:3488    1
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ["Infosys co-founder N S Raghavan's hotel venture Nadathur eyes 100 properties across Asia-Pacific"]
Actual label:2922    1
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['`Infosys should spin off low margin businessÃ¢??, says Pari Natarajan, CEO, Zinnov']
Actual label:436    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Infosys results throws up positive surprises: IIFL Institutional Equities Report
Actual label:547    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Marke

...


Actual Title: ['Buy Reliance Industries Ltd., target Rs 1180.0 :  Sacchitanand Uttekar']
Actual label:566    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Infosys pulls up the BSE IT index; TVS Electronics & HCL Tech gain
Actual label:1837    2
Name: Label, dtype: int64
Predicted label: 1

Actual Title: ['Tata Group set to enter ecommerce space through the launch of online marketplace CLiQ']
Actual label:1287    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: H-1B workers have minimal skills: Infosys whistleblower
Actual label:2952    2
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Sell Infosys Ltd., target Rs 1120: CapitalVia Global Research']
Actual label:1132    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Infosys denies governance lapses, rift between founders, board
Actual label:451    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Tata Motors makes Jaguar history in UK
Actual label:888    1
Name: Label


Actual Title: Sensex, Nifty weak; Tata Steel up 3% on high production
Actual label:2994    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Reliance Industries outbids GAIL to buy all its own CBM gas
Actual label:4338    2
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Buzzing stocks: Maruti Suzuki climbs, United Spirits drops, Reliance Defence zooms
Actual label:2090    2
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Volume growth in MHCV segment, JLR ex-China encouraging; buy Tata Motors with 2-year view: Mayuresh Joshi']
Actual label:2446    2
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Infosys wins multi-year contract from Deutsche Bank
Actual label:2891    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Maintaining margins should be a cakewalk for Infosys if they begin at 25% in FY15: Ankit Pandey']
Actual label:1418    2
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Stocks in news: RIL, Lupin

Predicted label: 2

Actual Title: Sensex, Nifty up 1%; BHEL, Tata Motors top gainers
Actual label:606    0
Name: Label, dtype: int64
Predicted label: 1

Actual Title: Nifty holds 8400, Sensex opens flat; BHEL, Tata Motors gainers
Actual label:3726    2
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Tata Steel falls over 2% on fall in Q1 numbers']
Actual label:3237    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Infosys up 1%; poised to achieve sales target
Actual label:4036    1
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Buy Reliance Power with a target of Rs 76: Prakash Gaba']
Actual label:1370    2
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Infosys slips 2% post AGM
Actual label:2594    2
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Indian tech companies like Infosys, TCS and others wary of government contracts']
Actual label:1420    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Nif

Actual Title: ['Monsoon deficit not to impact sales, says Tata Motors']
Actual label:2071    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Infosys, Cognizant plan big buybacks to cut cash pile']
Actual label:2879    1
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Buy Tata Motors DVR with a target of Rs 272: Mitesh Thacker']
Actual label:516    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ["Significant valuation re-rating likely as Reliance Jio's visibility grows: Jal Irani, Edelweiss"]
Actual label:2871    2
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Sensex, Nifty still in red; Coal India, ONGC, Reliance gainers
Actual label:1615    1
Name: Label, dtype: int64
Predicted label: 1

Actual Title: ['Buy Tata Consultancy Services, target Rs 2775:  Reliance Securities']
Actual label:2177    2
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Tata Steel statement following result of UK pension ballot-
Actual label:1

Actual Title: Retain buy on Tata Steel, India capacity likely to double in 5 years: Deutsche Bank
Actual label:1108    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Why it will be a tough task for Vishal Sikka to revive Infosys back to its glory']
Actual label:4127    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: Stocks in news: HDFC, TCS, ICICI Bank, Tata Motors, Jet
Actual label:82    0
Name: Label, dtype: int64
Predicted label: 2

Actual Title: ['Reliance ADAG stocks soar up to 14% on mega defence alliance buzz']
Actual label:3559    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Reduce ACC Ltd., target Rs 1550.0 :  Reliance Securities']
Actual label:2066    0
Name: Label, dtype: int64
Predicted label: 0

Actual Title: ['Sell  Tata Steel Ltd.  target Rs  451:   Motilal Oswal']
Actual label:1819    0
Name: Label, dtype: int64
Predicted label: 2

Actual Title: Maruti, Hero Moto, Tata Motors to get astro support: Gupta
Actual label:407